# MPQP Database Interaction

This notebook demonstrates how to manage quantum circuit execution results using the `mpqp` library. It includes setting up the database, running quantum circuits on simulators, storing results, and performing database queries.

In [1]:
from mpqp.all import *
from mpqp.execution.connection.env_manager import get_env_variable, save_env_variable

## Database Setup

In [2]:
from mpqp.local_storage.setup import setup_local_storage
from mpqp.local_storage.delete import clear_local_storage

To store and retrieve circuit execution results, we need a database. The `setup_local_storage` function initializes this database.

by default `setup_local_storage` will be called `quantum_jobs.local_storage`. We can change this by giving the database name as an argument
The name of the database will be stored in the global variable of mpqp.

In [3]:
save_local_storage = get_env_variable("DATA_BASE")
setup_local_storage()
print(get_env_variable("DATA_BASE"))

C:\Users\...\.mpqp\result_storage.db


In [4]:
setup_local_storage("test_database.db")
print(get_env_variable("DATA_BASE"))
clear_local_storage()

test_database.db


## Insert Results into the Database

To save circuit execution results, use the `insert_result` function from `mpqp.local_storage.save`.

In [5]:
from mpqp.local_storage.save import insert_results

We create a simple quantum circuit using gates and run it on a simulator.
We can then store the results in the database.

By default if a job already exists the result will put the same job_id it can be change by setup `compile_same_job` args in `insert_result` to `false`.

In [6]:
c = QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM")

result = run(c, device=IBMDevice.AER_SIMULATOR)

id = insert_results(result)
print(id)

[1]


We can also store a `BatchResult`.

In [7]:
result2 = run(c, device=[IBMDevice.AER_SIMULATOR, GOOGLEDevice.CIRQ_LOCAL_SIMULATOR])

id2 = insert_results(result2)
print(id2)

[2, 3]


In [8]:
c = QCircuit([H(0), BasisMeasure()], label="H BM")

result3 = run(c, device=[IBMDevice.AER_SIMULATOR, GOOGLEDevice.CIRQ_LOCAL_SIMULATOR])

id = insert_results(result3)
print(id)

[4, 5]


In [9]:
c1 = QCircuit([], nb_qubits=2)
c2 = QCircuit([Id(0), Id(1)], nb_qubits=2, label="Id")

result12 = run([c1, c2], device=IBMDevice.AER_SIMULATOR)

id = insert_results(result12)
print(id)

[6, 7]


## Query the Database

The `mpqp.local_storage.queries` module provides functions to fetch results and jobs.

In [10]:
from mpqp.local_storage.queries import *

assert isinstance(result, Result)
job = result.job

Retrieve all results stored in the database.

In [11]:
all_results = fetch_all_results()
for r in all_results:
    print(r)

{'id': 1, 'job_id': 1, 'data': '"[Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)]"', 'error': None, 'shots': 1024, 'created_at': '2025-02-24 16:27:52'}
{'id': 2, 'job_id': 1, 'data': '"[Sample(2, index=0, count=533, probability=0.5205078125), Sample(2, index=3, count=491, probability=0.4794921875)]"', 'error': None, 'shots': 1024, 'created_at': '2025-02-24 16:27:53'}
{'id': 3, 'job_id': 2, 'data': '"[Sample(2, index=0, count=546, probability=0.533203125), Sample(2, index=3, count=478, probability=0.466796875)]"', 'error': None, 'shots': 1024, 'created_at': '2025-02-24 16:27:53'}
{'id': 4, 'job_id': 3, 'data': '"[Sample(1, index=0, count=526, probability=0.513671875), Sample(1, index=1, count=498, probability=0.486328125)]"', 'error': None, 'shots': 1024, 'created_at': '2025-02-24 16:27:53'}
{'id': 5, 'job_id': 4, 'data': '"[Sample(1, index=0, count=478, probability=0.466796875), Sample(1, index=1, count=546, probability=

Retrieve all jobs stored in the database.


In [12]:
all_jobs = fetch_all_jobs()
for j in all_jobs:
    print(j)

{'id': 1, 'type': 'SAMPLE', 'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label=\\"H CX BM\\")"', 'device': 'IBMDevice.AER_SIMULATOR', 'measure': '"BasisMeasure([0, 1], c_targets=[0, 1])"', 'remote_id': None, 'status': None, 'created_at': '2025-02-24 16:27:52'}
{'id': 2, 'type': 'SAMPLE', 'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label=\\"H CX BM\\")"', 'device': 'GOOGLEDevice.CIRQ_LOCAL_SIMULATOR', 'measure': '"BasisMeasure([0, 1], c_targets=[0, 1])"', 'remote_id': None, 'status': None, 'created_at': '2025-02-24 16:27:53'}
{'id': 3, 'type': 'SAMPLE', 'circuit': '"QCircuit([H(0), BasisMeasure()], label=\\"H BM\\")"', 'device': 'IBMDevice.AER_SIMULATOR', 'measure': '"BasisMeasure([0], c_targets=[0])"', 'remote_id': None, 'status': None, 'created_at': '2025-02-24 16:27:53'}
{'id': 4, 'type': 'SAMPLE', 'circuit': '"QCircuit([H(0), BasisMeasure()], label=\\"H BM\\")"', 'device': 'GOOGLEDevice.CIRQ_LOCAL_SIMULATOR', 'measure': '"BasisMeasure([0], c_targets=[0])"'

Retrieve specific result using their ID.

In [13]:
local_storage_result_id_1 = fetch_results_with_id(1)
local_storage_result_id_1

[{'id': 1,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2025-02-24 16:27:52'}]

Retrieve specific job using their ID.

In [14]:
assert local_storage_result_id_1 is not None
local_storage_job_id_1 = fetch_jobs_with_id(local_storage_result_id_1[0]['job_id'])
local_storage_job_id_1

[{'id': 1,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure([0, 1], c_targets=[0, 1])"',
  'remote_id': None,
  'status': None,
  'created_at': '2025-02-24 16:27:52'}]

Retrieve all results using the job ID.

In [15]:
local_storage_jobs_with_results = fetch_results_with_job_id(1)
local_storage_jobs_with_results

[{'id': 1,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2025-02-24 16:27:52'},
 {'id': 2,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=533, probability=0.5205078125), Sample(2, index=3, count=491, probability=0.4794921875)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2025-02-24 16:27:53'}]

Retrieve all jobs associated with a job.

In [16]:
local_storage_jobs_with_job = fetch_jobs_with_job(job)
local_storage_jobs_with_job

[{'id': 1,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure([0, 1], c_targets=[0, 1])"',
  'remote_id': None,
  'status': None,
  'created_at': '2025-02-24 16:27:52'}]

Retrieve all jobs associated with a result.

In [17]:
local_storage_jobs_with_results = fetch_jobs_with_result(result)
local_storage_jobs_with_results

[{'id': 1,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure([0, 1], c_targets=[0, 1])"',
  'remote_id': None,
  'status': None,
  'created_at': '2025-02-24 16:27:52'}]

Retrieve all results associated with a result.

In [18]:
local_storage_results_with_results = fetch_results_with_result(result)
local_storage_results_with_results

[{'id': 1,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2025-02-24 16:27:52'}]

Retrieve all results associated with a result and result.job.

In [19]:
local_storage_results_with_results_and_job = fetch_results_with_result_and_job(result)
local_storage_results_with_results_and_job

[{'id': 1,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2025-02-24 16:27:52'}]

## Export Data

You can also export database entries using the `mpqp.local_storage.local_storage_export` module.

In [20]:
from mpqp.local_storage.load import *

transform a local_storage format jobs to mpqp format.

In [21]:
jobs_local_storage_to_mpqp(local_storage_job_id_1)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1]))]

transform a local_storage format results to mpqp format.

In [22]:
print(results_local_storage_to_mpqp(local_storage_result_id_1)[0])

Result: H CX BM, IBMDevice, AER_SIMULATOR
 Counts: [515, 0, 0, 509]
 Probabilities: [0.50293, 0, 0, 0.49707]
 Samples:
  State: 00, Index: 0, Count: 515, Probability: 0.5029297
  State: 11, Index: 3, Count: 509, Probability: 0.4970703
 Error: None


Fetch all jobs

In [23]:
get_all_jobs()

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])),
 Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), GOOGLEDevice.CIRQ_LOCAL_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])),
 Job(JobType.SAMPLE, QCircuit([H(0), BasisMeasure()], label="H BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0], c_targets=[0])),
 Job(JobType.SAMPLE, QCircuit([H(0), BasisMeasure()], label="H BM"), GOOGLEDevice.CIRQ_LOCAL_SIMULATOR, BasisMeasure([0], c_targets=[0])),
 Job(JobType.STATE_VECTOR, QCircuit(2, label="circuit 1"), IBMDevice.AER_SIMULATOR),
 Job(JobType.STATE_VECTOR, QCircuit([Id(0), Id(1)], nb_qubits=2, label="Id"), IBMDevice.AER_SIMULATOR)]

Fetch all results

In [24]:
get_all_results()

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=533, probability=0.5205078125), Sample(2, index=3, count=491, probability=0.4794921875)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), GOOGLEDevice.CIRQ_LOCAL_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=546, probability=0.533203125), Sample(2, index=3, count=478, probability=0.466796875)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), BasisMeasure()], label="H BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0], c_targets=[0]))

In [25]:
get_jobs_with_job(job)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1]))]

In [26]:
get_jobs_with_result(result)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1]))]

In [27]:
get_results_with_result_and_job(result)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)], None, 1024)]

In [28]:
get_results_with_result(result)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)], None, 1024)]

In [29]:
get_results_with_id(1)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)], None, 1024)]

In [30]:
get_jobs_with_id(1)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1]))]

In [31]:
get_results_with_job_id(1)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=515, probability=0.5029296875), Sample(2, index=3, count=509, probability=0.4970703125)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure([0, 1], c_targets=[0, 1])), [Sample(2, index=0, count=533, probability=0.5205078125), Sample(2, index=3, count=491, probability=0.4794921875)], None, 1024)]

## Remove elements

In [32]:
from mpqp.local_storage.setup import *
from mpqp.local_storage.delete import *

In [33]:
print(len(get_results_with_id(4)))
remove_results_with_id(4)
print(len(get_results_with_id(4)))

1
0


In [34]:
print(len(get_jobs_with_id(4)))
remove_jobs_with_id(4)
print(len(get_jobs_with_id(4)))

1
0


In [35]:
print(len(get_results_with_job_id(1)))
remove_all_with_job_id(1)
print(len(get_results_with_job_id(1)))

2
0


In [36]:
print(len(fetch_results_with_id(5)))
remove_results_with_results_local_storage(fetch_results_with_id(5))
print(len(fetch_results_with_id(5)))

1
0


In [37]:
clear_local_storage()
save_env_variable("DATA_BASE", save_local_storage)

True